In [22]:
import pandas as pd

In [23]:
train_set = pd.read_csv('./dataset/train.csv')
test_set = pd.read_csv('./dataset/test.csv')
train_label = train_set['label']
train_set = train_set.drop('label', axis=1)

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1, 
                out_channels=6, 
                kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d((2,2))
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels=6, 
                out_channels=16, 
                kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d((2,2)),
        )
        self.output = nn.Sequential(
            nn.Linear(256, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10),
        )
    
    def forward(self, input):
        x = self.conv1(input)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        return self.output(x)

In [25]:
# 检查 MPS 可用性
if torch.backends.mps.is_available():
    device = torch.device("mps")  # Apple GPU
else:
    device = torch.device("cpu")  # 回退到 CPU
print(f"Using device: {device}")

Using device: mps


In [26]:
cnn = Net()
cnn.to(device)
print(cnn)

optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

for epoch in range(10):
    for i in range(0, len(train_set), 100):
        x = torch.tensor(train_set[i:i+100].values, dtype=torch.float32).view(-1, 1, 28, 28).to(device)
        y = torch.tensor(train_label[i:i+100].values, dtype=torch.long).to(device)
        optimizer.zero_grad()
        output = cnn(x)
        loss = loss_func(output, y)
        loss.backward()
        optimizer.step()
        print(f'epoch: {epoch}, loss: {loss.item()}')

Net(
  (conv1): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (output): Sequential(
    (0): Linear(in_features=256, out_features=120, bias=True)
    (1): ReLU()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): ReLU()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)
epoch: 0, loss: 6.3087921142578125
epoch: 0, loss: 4.404211044311523
epoch: 0, loss: 3.5492706298828125
epoch: 0, loss: 3.030129909515381
epoch: 0, loss: 3.4352681636810303
epoch: 0, loss: 2.9160759449005127
epoch: 0, loss: 2.633502721786499
epoch: 0, loss: 2.0943965911865234
epoch: 0, loss: 1.7218209505081177
epoch: 0, loss: 1.8383809328079224
epoch: 0, loss

In [27]:
from PIL import Image
import numpy as np

img = train_set.loc[10].values.reshape(28, 28)
y = train_label.loc[10]
print(y)
Image.fromarray(np.uint8(img))

8


In [28]:
cnn(torch.tensor(img.reshape(28*28), dtype=torch.float32).view(-1, 1, 28, 28).to(device))


tensor([[-4.4736, -1.2054, -0.3259,  1.9120, -6.1980,  0.0375, -3.9605, -8.1354,
         18.9155, -6.4407]], device='mps:0', grad_fn=<LinearBackward0>)

In [29]:
result = cnn(torch.tensor(test_set.values, dtype=torch.float32).view(-1, 1, 28, 28).to(device))

In [30]:
result = torch.argmax(result, dim=1)

In [32]:
with open('./dataset/result.csv', 'w') as f:
    f.write('ImageId,Label\n')
    for i, v in enumerate(result):
        f.write(f'{i+1},{v}\n')